# Computational Project Part 2

In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as la

In [2]:
def predictor_corrector (x_0, pi_0, z_0, Q, c, A, b):
    
    ################# step 0 #################
    
    # Prepare data
    e =np.ones([3,1]) 
    
    x_list = [x_0]
    cTx_list = [(c.T @ x_0) [0,0]]
    
    pi_list = [pi_0]
    z_list = [z_0]
    bTpi_list = [(b.T @ pi_0) [0,0]]
    
    tau_list = [' ']
    residual_1_list = [np.linalg.norm(A@x_0 - b)]
    residual_2_list = [np.linalg.norm(-Q.T @ x_0 + A.T@pi_0 + z_0 - c)]
    residual_3_list = [np.linalg.norm(x_0.T @ z_0)]
    
    # Start with an infeasible solution
    x = x_0
    pi = pi_0
    z = z_0
    
    
    # Set tolerance
    epsilon = 1e-08  
     
    for n in range(30): 
      
        if (np.linalg.norm(A@x-b) <= epsilon) and (np.linalg.norm(-Q.T @ x + A.T@pi + z -c) <= epsilon) and (np.linalg.norm(x.T @ z) <= epsilon):
            print ("Stop at iteration: ", n)
            break
         
        else: 
            
            ##################### step 1 ######################
            rp = A @ x - b
            rd = - Q.T @ x + A.T @ pi + z - c
            X = np.diag(x.T[0])
            Z = np.diag(z.T[0])
            XZe = X @ Z @ e
            
             
            # construct b
            rs_b = np.concatenate((-rd, -rp, -XZe), axis=0)
            # construct A
            row_1 = np.concatenate((-Q, A.T, np.eye(3)), axis=1)
            row_2 = np.concatenate((A, np.zeros([2,2]), np.zeros([2,3])), axis=1) 
            row_3 = np.concatenate((Z, np.zeros([3,2]), X), axis=1)
            ls_A = np.concatenate((row_1, row_2, row_3), axis=0) 
            # solve Ax = b
            ls_x = la.solve(ls_A, rs_b)
            
            # compute affine scaling direction
            affd_x = ls_x[:3]
            affd_pi = ls_x[3:5]
            affd_z = ls_x[5:]

            ###################### step 2 ######################

            temp_list_1 = []
            for i in range(len(affd_x)):
                if affd_x[i] < 0:
                    temp_list_1.append( - x[i]/ affd_x[i])
             
            temp_list_2 = []
            for i in range(len(affd_z)):
                if affd_z[i] < 0:
                    temp_list_2.append( - z[i]/ affd_z[i])
             
            # compute step size for affine scaling direction
            aff_alpha = min(1,min(temp_list_1), min(temp_list_2))
             
            y = x.T @ z / 3
            aff_y = (x + aff_alpha  * affd_x).T @ (z + aff_alpha  * affd_z) / 3
            tau = (aff_y / y) ** 3
            
            # construct b  
            D_x = np.diag(affd_x.T[0])
            D_z = np.diag(affd_z.T[0])
            # add corrector
            rs_b = np.concatenate((- rd, - rp, - XZe - D_x @ D_z @ e + tau * y), axis=0)    
            # solve Ax = b
            ls_x = la.solve(ls_A, rs_b)
            
            # compute direction
            d_x = ls_x[:3]
            d_pi = ls_x[3:5]
            d_z = ls_x[5:]
 
            ##################### step 3 ######################

            temp_list = []
            for i in range(len(d_x)):
                if d_x[i] < 0:
                    temp_list.append( - x[i]/ d_x[i])
            alpha_x_max = min(1, min (temp_list))  
             

            temp_list = []
            for i in range(len(d_z)):
                if d_z[i] < 0:
                    temp_list.append( - z[i]/ d_z[i])
            alpha_z_max = min(1, min (temp_list))  
            
            # compute step size 
            alpha = min(1, 0.95 * alpha_x_max, 0.95 * alpha_z_max)
 
            
            # update
            x = x + alpha * d_x 
            pi = pi + alpha * d_pi 
            z = z + alpha * d_z 
           
            # Primal solution
            x_list.append(x)
            cTx_list.append ((c.T @ x) [0,0])
            
            # Dual solution
            pi_list.append(pi)
            z_list.append(z)
            bTpi_list.append((b.T @ pi) [0,0])
            
            # Residual and tau
            tau_list.append (tau[0,0])
            residual_1_list.append(np.linalg.norm(A@x-b))
            residual_2_list.append(np.linalg.norm(-Q.T @ x + A.T@pi + z - c))
            residual_3_list.append((x.T @ z)[0,0])
            
    return (x_list, cTx_list, pi_list, z_list, bTpi_list, tau_list, residual_1_list, residual_2_list, residual_3_list)


In [3]:
Q = np.array([[0.02778, 0.00387, 0.00021],
              [0.00387, 0.01112, -0.0002],
              [0.00021, -0.0002, 0.00115]])
c = np.array([[0],[0],[0]])
A = np.array([[0.1073, 0.0737, 0.0627],
              [1,1,1]])
b = np.array([[0.0650], [1]])

Start with an infeasible solution

In [4]:
x_0 = np.array([[1],[1],[1]])
pi_0 = np.array([[1],[1]])
z_0 = np.array([[1],[1],[1]])

In [5]:
x_list, cTx_list, pi_list, z_list, bTpi_list, tau_list, residual_1_list, residual_2_list, residual_3_list = predictor_corrector (x_0, pi_0, z_0, Q, c, A, b)  

Stop at iteration:  8


* Primal solution

In [6]:
x1_list = [i[0,0] for i in x_list]
x2_list = [i[1,0] for i in x_list]
x3_list = [i[2,0] for i in x_list]

In [7]:
pd.set_option('display.float_format', lambda x: '%.9f' % x)

In [8]:
primal_solution = {'$x_1^{(k)}$': x1_list,
                   '$x_2^{(k)}$': x2_list,
                   '$x_3^{(k)}$': x3_list,
                   '$c^T x$': cTx_list}
pd.DataFrame (primal_solution) 

,$x_1^{(k)}$,$x_2^{(k)}$,$x_3^{(k)}$,$c^T x$
0,1.000000000,1.000000000,1.000000000,0.000000000
1,0.050000000,0.448614116,0.683928296,0.000000000
2,0.017312985,0.169889624,0.825590825,0.000000000
3,0.015198689,0.149016882,0.836424100,0.000000000
4,0.015894912,0.144761318,0.839392084,0.000000000
5,0.022028593,0.119789584,0.858187852,0.000000000
6,0.025939248,0.103919780,0.870141274,0.000000000
7,0.026285546,0.102515005,0.871199464,0.000000000
8,0.026303047,0.102444010,0.871252944,0.000000000


* Dual solution

In [9]:
pi1_list = [i[0,0] for i in pi_list]
pi2_list = [i[1,0] for i in pi_list]

z1_list = [i[0,0] for i in z_list]
z2_list = [i[1,0] for i in z_list]
z3_list = [i[2,0] for i in z_list]

In [10]:
dual_solution = {'$\pi_1^{(k)}$': pi1_list,
                 '$\pi_2^{(k)}$': pi2_list,
                 '$z_1^{(k)}$': z1_list,
                 '$z_2^{(k)}$': z2_list,
                 '$z_3^{(k)}$': z3_list,
                 '$b^T \pi$': bTpi_list}
pd.DataFrame (dual_solution) 
# .style.hide_index()

,$\pi_1^{(k)}$,$\pi_2^{(k)}$,$z_1^{(k)}$,$z_2^{(k)}$,$z_3^{(k)}$,$b^T \pi$
0,1.000000000,1.000000000,1.000000000,1.000000000,1.000000000,1.065000000
1,-20.206745134,1.258530294,1.102350132,0.423671324,0.197299163,-0.054908140
2,-16.968481268,1.068164992,0.767140851,0.197373397,0.009864958,-0.034786290
3,-1.104876954,0.070352951,0.050038710,0.013283595,0.000517464,-0.001464051
4,-0.049884829,0.004078903,0.002501935,0.001150727,0.000038364,0.000836389
5,0.003776684,0.000731681,0.000125097,0.000241857,0.000005320,0.000977166
6,0.006962107,0.000548763,0.000010007,0.000020385,0.000000349,0.001001300
7,0.007231352,0.000533489,0.000000500,0.000001028,0.000000017,0.001003527
8,0.007244848,0.000532724,0.000000025,0.000000051,0.000000001,0.001003639


* Residual and $\tau$ 

In [11]:
pd.reset_option('display.float_format')

In [12]:
tau_residual = {'$\tau$': tau_list,
                '$||Ax-b ||$': residual_1_list,
                'residual2': residual_2_list,
                '$(x^{(k)})^T z$': residual_3_list}
pd.DataFrame (tau_residual)
# .style.hide_index()

,$\tau$,$||Ax-b ||$,residual2,$(x^{(k)})^T z$
0,,2.007968e+00,3.577221e+00,3.000000e+00
1,0.00589407,1.832696e-01,3.264973e-01,3.801209e-01
2,0.00662218,1.284440e-02,2.288247e-02,5.495761e-02
3,0.00114608,6.422200e-04,1.144123e-03,3.172822e-03
4,1.96069e-05,4.850730e-05,8.641640e-05,2.385510e-04
5,0.0104547,6.053190e-06,1.078384e-05,3.629293e-05
6,0.00690251,3.026595e-07,5.391919e-07,2.681929e-06
7,1.1781e-05,1.513756e-08,2.696776e-08,1.337910e-07
8,1.7116e-09,7.568802e-10,1.348392e-09,6.686523e-09
